# Data Understanding

Nesta etapa buscamos desenvolver um entendimento profundo dos dados disponibilizados pelo Portal da Transparência, avaliando sua **estrutura**, **qualidade**, **completude** e **limitações**. Esta é uma fase essencial do CRISP-DM, pois fornece a base para um processo de preparação de dados consistente, reprodutível e confiável.

Nesta fase **não realizamos nenhum tipo de limpeza ou transformação** — o foco é compreender o estado bruto da informação e responder questões como:

- **Quais dados estão disponíveis e como estão organizados?**
- **Qual é a estrutura das tabelas e seus tipos de variáveis?**
- **Existem valores ausentes, inconsistências ou anomalias?**
- **Há informações sensíveis que exigem atenção do ponto de vista ético e da LGPD?**
- **Quais limitações podem impactar as etapas de preparação e análise?**

O resultado desta etapa é um panorama claro do conjunto de dados, permitindo antecipar desafios e orientar decisões de preparação e modelagem nas fases subsequentes.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
# Definindo quantas colunas o pandas irá exibir
pd.set_option("display.max_columns", 100)

# Definindo como números float serão exibidos (2 casas decimais)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

- Lendo os arquivos CSV

In [3]:
DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

len(csv_files), csv_files

(12,
 [WindowsPath('../data/raw/abr2025.csv'),
  WindowsPath('../data/raw/ago2025.csv'),
  WindowsPath('../data/raw/dez2025.csv'),
  WindowsPath('../data/raw/fev2025.csv'),
  WindowsPath('../data/raw/jan2025.csv'),
  WindowsPath('../data/raw/jul2025.csv'),
  WindowsPath('../data/raw/jun2025.csv'),
  WindowsPath('../data/raw/mai2025.csv'),
  WindowsPath('../data/raw/mar2025.csv'),
  WindowsPath('../data/raw/nov2025.csv'),
  WindowsPath('../data/raw/out2025.csv'),
  WindowsPath('../data/raw/set2025.csv')])

In [4]:
dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df_raw = pd.concat(dfs, ignore_index=True)

df_raw.shape

(24009, 16)

Após a importação de todos os arquivos `.csv` referentes aos meses de 2025 e a posterior concatenação em um único DataFrame, obtivemos um conjunto contendo **24.009 linhas** e **16 colunas**.

Esse volume representa a totalidade dos registros mensais de remuneração dos servidores públicos no período analisado e constitui a base bruta sobre a qual conduziremos as avaliações de estrutura, qualidade e consistência dos dados.

## Observações sobre Codificação dos Arquivos

Durante a etapa de ingestão dos dados, foi identificado que os arquivos CSV não utilizavam codificação **UTF-8**, o que gerou erros de leitura e problemas relacionados a caracteres acentuados.

Após testes e verificação manual, a codificação **`latin1`** mostrou-se compatível com todos os arquivos. Esse padrão é comum em **sistemas legados da administração pública brasileira**, e sua adoção garantiu a leitura correta dos caracteres especiais, preservando a integridade textual das informações.


In [5]:
df_raw.head()

,Referência,Nome,Cargo,Data Admissão,Tipo de Regime,Descontos,Liquido,Data Desligamento,Proventos,Contrato,Atividade,Nome Atividade,Tipo de Contrato,Data Prevista Termino Contrato,Carga Horária (Sem.),arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,30.00,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.00,NaN,NaN,Efetivo,NaN,30.00,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,abr2025.csv


In [6]:
df_raw.tail()

,Referência,Nome,Cargo,Data Admissão,Tipo de Regime,Descontos,Liquido,Data Desligamento,Proventos,Contrato,Atividade,Nome Atividade,Tipo de Contrato,Data Prevista Termino Contrato,Carga Horária (Sem.),arquivo_origem
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,40.00,set2025.csv
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.00,NaN,NaN,Efetivo,NaN,40.00,set2025.csv
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.00,NaN,NaN,Efetivo,NaN,30.00,set2025.csv
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.00,NaN,NaN,Efetivo,NaN,30.00,set2025.csv
24008,NaN,NaN,NaN,NaN,NaN,"918.286,26","4.302.011,08",NaN,"5.220.297,34",NaN,NaN,NaN,NaN,NaN,NaN,set2025.csv


## Dicionário de Dados

A seguir, uma descrição detalhada das principais variáveis presentes no conjunto de dados original disponibilizado pelo Portal da Transparência:

- **Referência**: Mês e tipo de processamento da folha de pagamento.
- **Nome**: Nome completo do servidor público (será posteriormente anonimitado).
- **Cargo**: Cargo ocupado pelo servidor no período analisado.
- **Data Admissão**: Data de ingresso do servidor no quadro da prefeitura.
- **Tipo de Regime**: Regime previdenciário ao qual o servidor está vinculado.
- **Descontos**: Valor total dos descontos aplicados ao servidor.
- **Líquido**: Valor líquido recebido após a aplicação dos descontos.
- **Data Desligamento**: Data de desligamento do servidor, quando existente.
- **Proventos**: Valor bruto da remuneração antes dos descontos.
- **Contrato**: Identificador interno do contrato no sistema de gestão municipal.
- **Atividade**: Código interno utilizado para classificar atividades ou funções.
- **Nome Atividade**: Descrição textual da atividade exercida.
- **Tipo de Contrato**: Modalidade de vínculo do servidor com a prefeitura (efetivo, comissionado etc.).
- **Data Prevista Término Contrato**: Data prevista para encerramento do contrato, quando aplicável.
- **Carga Horária (Sem.)**: Jornada semanal de trabalho do servidor, em horas.

**Observação:** A última linha de cada arquivo mensal representa um **registro de totalização**, contendo a soma das colunas **Descontos**, **Líquido** e **Proventos** daquele mês.  
Esses registros não correspondem a servidores individuais e serão removidos na etapa de **Data Preparation**, a fim de preservar a integridade analítica do conjunto de dados.

- Foi realizada uma leitura apenas dos cabeçalhos de todos os arquivos CSV, permitindo mapear quais colunas estão presentes em cada arquivo e construir um dicionário que relaciona cada mês ao seu respectivo conjunto de variáveis.

In [7]:
schemas = {
    file.name: set(pd.read_csv(file, encoding="latin1", sep=";", nrows=0).columns)
    for file in csv_files
}

schemas

{'abr2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'ago2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'dez2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'Data Prevista Termino Contrato',
  'Descontos',
  'Liquido',
  'Nome',
  'Nome Atividade',
  'Proventos',
  'Referência',
  'Tipo de Contrato',
  'Tipo de Regime'},
 'fev2025.csv': {'Atividade',
  'Carga Horária (Sem.)',
  'Cargo',
  'Contrato',
  'Data Admissão',
  'Data Desligamento',
  'D

- Identificação do número de diferentes estruturas de colunas existentes entre os arquivos CSV, a fim de verificar possíveis variações de esquema ao longo dos meses.

In [8]:
unique_schemas = set(tuple(sorted(cols)) for cols in schemas.values())
len(unique_schemas)

1

Todos os arquivos apresentam exatamente o mesmo *schema*, não havendo variação na estrutura de colunas entre os meses analisados.

- Colunas do dataframe

In [9]:
df_raw.columns

Index(['Referência', 'Nome', 'Cargo', 'Data Admissão', 'Tipo de Regime',
       'Descontos', 'Liquido', 'Data Desligamento', 'Proventos', 'Contrato',
       'Atividade', 'Nome Atividade', 'Tipo de Contrato',
       'Data Prevista Termino Contrato', 'Carga Horária (Sem.)',
       'arquivo_origem'],
      dtype='object')

## Análise de Valores Ausentes (*Missing Values*)

A seguir é apresentada a proporção de valores ausentes por coluna, calculada como a razão entre o número de registros nulos e o total de linhas do dataset.

Essa verificação é fundamental para avaliar a **qualidade inicial dos dados**, identificar possíveis inconsistências e determinar quais variáveis exigem tratamento específico nas etapas de **Data Preparation** e **Exploratory Data Analysis**.

In [10]:
(df_raw.isna().sum() / len(df_raw)).sort_values(ascending=False)

Data Prevista Termino Contrato   1.00
Nome Atividade                   1.00
Atividade                        1.00
Data Desligamento                0.99
Descontos                        0.50
Proventos                        0.00
Tipo de Contrato                 0.00
Referência                       0.00
Nome                             0.00
Tipo de Regime                   0.00
Cargo                            0.00
Data Admissão                    0.00
Carga Horária (Sem.)             0.00
Contrato                         0.00
Liquido                          0.00
arquivo_origem                   0.00
dtype: float64

### Principais Observações

- As colunas **Data Prevista Término Contrato**, **Atividade** e **Nome Atividade** apresentam **100% de valores ausentes**, indicando ausência total de informação útil para este conjunto de dados. Por esse motivo, elas **não serão consideradas nas análises subsequentes**.

- A coluna **Data Desligamento** possui aproximadamente **99% de valores ausentes**, o que é compatível com o contexto de servidores públicos, considerando que a maior parte dos vínculos permanece ativa ao longo de 2025.

- A coluna **Descontos** apresenta cerca de **50% de valores ausentes**, o que é explicado pela natureza dos registros:
  
  - Entradas referentes a **vale-alimentação** não possuem descontos associados.  
  - Registros da **folha mensal** apresentam valores preenchidos.

- As demais variáveis — como **Proventos**, **Líquido**, **Cargo**, **Nome**, **Tipo de Contrato**, **Data Admissão**, entre outras — não apresentam valores ausentes, o que denota **boa consistência estrutural** do dataset.

---

#### Conclusão

De maneira geral, o conjunto de dados apresenta **boa qualidade**, com valores ausentes concentrados em colunas que:

- não são relevantes para os objetivos analíticos do projeto, ou  
- possuem ausência plenamente explicável pelo contexto administrativo.

Essas observações orientam a etapa de **Data Preparation**, garantindo que o tratamento dos dados seja conduzido de forma precisa e alinhada à natureza do dataset.

## Tipos de dados

In [11]:
df_raw.dtypes

Referência                         object
Nome                               object
Cargo                              object
Data Admissão                      object
Tipo de Regime                     object
Descontos                          object
Liquido                            object
Data Desligamento                  object
Proventos                          object
Contrato                          float64
Atividade                         float64
Nome Atividade                     object
Tipo de Contrato                   object
Data Prevista Termino Contrato    float64
Carga Horária (Sem.)              float64
arquivo_origem                     object
dtype: object

In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24009 entries, 0 to 24008
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Referência                      23997 non-null  object 
 1   Nome                            23997 non-null  object 
 2   Cargo                           23997 non-null  object 
 3   Data Admissão                   23997 non-null  object 
 4   Tipo de Regime                  23997 non-null  object 
 5   Descontos                       12045 non-null  object 
 6   Liquido                         24009 non-null  object 
 7   Data Desligamento               157 non-null    object 
 8   Proventos                       23931 non-null  object 
 9   Contrato                        23997 non-null  float64
 10  Atividade                       24 non-null     float64
 11  Nome Atividade                  24 non-null     object 
 12  Tipo de Contrato                

### Estrutura do Dataset e Tipos de Dados

A inspeção dos tipos de dados (`dtypes`) e das informações gerais do DataFrame (`info`) permite compreender a estrutura inicial do dataset, identificar inconsistências de tipagem e antecipar necessidades de tratamento para a etapa de *Data Preparation*.

#### Principais pontos observados

- O dataset contém **24.009 registros** e **16 colunas**, ocupando aproximadamente **3 MB de memória**, o que caracteriza um conjunto leve e de fácil manipulação.

- A maior parte das colunas está representada como `object`, incluindo:
  - Campos textuais (**Nome**, **Cargo**, **Tipo de Contrato**)
  - Campos monetários (**Proventos**, **Descontos**, **Líquido**)
  - Campos temporais (**Data Admissão**, **Data Desligamento**)

- Variáveis originalmente numéricas ou categóricas aparecem como `object` devido a:
  - Uso de **formatação brasileira** em valores monetários (`1.234,56`)
  - Datas armazenadas como texto em formatos não padronizados
  - Presença de valores ausentes que impedem inferência automática de tipo

- As colunas **Contrato**, **Atividade**, **Data Prevista Término Contrato** e **Carga Horária (Sem.)** aparecem como `float64`, embora algumas representem **códigos**, **identificadores** ou **valores inteiros**. Essas tipagens serão revisadas e corrigidas na etapa de preparação dos dados.

---

#### Conclusão

A estrutura geral do dataset é consistente, mas exige os seguintes ajustes:

- Conversão correta de valores monetários para `float` usando padrão numérico internacional.
- Padronização e conversão de campos de data para o tipo `datetime64`.
- Avaliação e possível remoção de colunas com baixa taxa de preenchimento ou irrelevantes para o objetivo analítico.

Essas observações guiarão a próxima fase do CRISP-DM, garantindo que o dataset final seja limpo, coerente e adequado para análises e modelagens subsequentes.

## Estatísticas Descritivas das Colunas Numéricas

A análise das estatísticas descritivas das variáveis numéricas permite compreender a **distribuição**, **tendência central** e **dispersão** dos valores presentes no dataset. Essa etapa é fundamental para identificar colunas com baixa variabilidade, possíveis inconsistências e oportunidades de transformação ou normalização durante o processo de preparação dos dados.

In [13]:
df_raw.describe()

,Contrato,Atividade,Data Prevista Termino Contrato,Carga Horária (Sem.)
count,"23,997.00",24.00,0.00,"23,997.00"
mean,1.05,10.00,NaN,35.34
std,0.22,0.00,NaN,7.48
min,1.00,10.00,NaN,6.00
25%,1.00,10.00,NaN,30.00
50%,1.00,10.00,NaN,40.00
75%,1.00,10.00,NaN,40.00
max,2.00,10.00,NaN,44.00


#### Principais Observações

- **Contrato**
  - Possui **23.997 registros válidos**, com valores variando apenas entre **1.0 e 2.0**.
  - Apresenta **baixa variabilidade**, com média próxima de **1.05**.
  - Indica tratar-se de um **código interno do sistema**, sem utilidade analítica direta neste momento.

- **Atividade**
  - Apenas **24 registros preenchidos** em todo o dataset.
  - Valor constante (**10.0**) e sem qualquer variação.
  - Não apresenta potencial informativo, reforçando sua exclusão nas próximas etapas.

- **Data Prevista Término Contrato**
  - **Nenhum registro preenchido** (`count = 0`).
  - A ausência total de dados inviabiliza qualquer tipo de análise.
  - Será descartada nas etapas de preparação.

- **Carga Horária (Sem.)**
  - Possui **23.997 registros válidos**, com valores entre **6 e 44 horas semanais**.
  - Mediana de **40 horas**, indicando jornada padrão predominante.
  - Variável **relevante** para comparações entre cargos, categorias e tipos de vínculo.

---

#### Conclusão

A análise estatística mostra que:

- Algumas variáveis numéricas correspondem apenas a **códigos internos** ou apresentam **variabilidade nula**, não contribuindo para objetivos analíticos.
- A coluna **Carga Horária (Sem.)** destaca-se como uma das variáveis mais úteis, com potencial para análises comparativas e insights estruturais.
- As colunas **Atividade** e **Data Prevista Término Contrato** não agregam valor ao projeto e podem ser removidas ou ignoradas nas etapas subsequentes de preparação e análise.

## Inspeção inicial das colunas de data

In [14]:
df_raw["Data Admissão"].head()

0    08/04/1991
1    08/04/1991
2    04/12/2023
3    04/12/2023
4    08/04/1991
Name: Data Admissão, dtype: object

In [15]:
df_raw["Data Desligamento"].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Data Desligamento, dtype: object

#### Observações

- **Data Admissão**
  - Os valores estão armazenados como texto (`object`), seguindo o padrão **brasileiro** `DD/MM/AAAA`.
  - A coluna apresenta **alto grau de preenchimento**, indicando boa confiabilidade.
  - Não há informações de horário — apenas a data de ingresso do servidor.

- **Data Desligamento**
  - A maior parte dos registros está nula (`NaN`), o que é esperado, pois indica servidores **ainda ativos**.
  - Quando presente, representa a data formal de desligamento do servidor.
  - A coluna possui significado semântico importante: **nulo ≠ dado ausente**, e sim “vínculo ativo”.

---

#### Implicações para o Tratamento dos Dados

- Ambas as colunas exigem **conversão para o tipo `datetime64`** e padronização de formato.
- A coluna **Data Desligamento** requer tratamento diferenciado, pois valores nulos carregam **informação contextual importante**.
- Decisões relacionadas à conversão, normalização e criação de variáveis derivadas (ex.: tempo de casa) serão aplicadas na etapa de **Data Preparation**.

## Resumo da Compreensão dos Dados

Nesta etapa de **Data Understanding**, foi conduzida uma análise exploratória inicial com foco na **estrutura**, **qualidade** e **consistência** dos dados provenientes do Portal da Transparência.

### Principais pontos consolidados

- Os dados estão distribuídos em múltiplos arquivos CSV mensais, todos compartilhando um **schema consistente**, o que permite a concatenação sem conflitos.
- A codificação dos arquivos é **`latin1`**, apropriada para lidar com caracteres especiais presentes nos sistemas legados da administração pública.
- O dataset consolidado possui **24.009 registros** e **16 colunas**, totalizando aproximadamente **3 MB**.
- A inspeção dos tipos (`dtypes`) mostrou que diversas colunas estão tipadas como `object`, incluindo:
  - Valores monetários,
  - Datas,
  - Variáveis categóricas.

- A formatação brasileira de números e datas justifica a tipagem textual no estado bruto.
- A análise de valores ausentes revelou:
  - Colunas amplamente completas (como **Nome**, **Cargo**, **Proventos**).
  - Colunas com alta taxa de nulos, como **Data Desligamento**, **Atividade**, **Nome Atividade** e **Data Prevista Término Contrato**, cujas ausências foram interpretadas de maneira contextual.

- Todas as colunas de data encontram-se no formato textual `DD/MM/AAAA`.
- A análise estatística das variáveis numéricas mostrou valores coerentes com a realidade administrativa municipal (como a **carga horária semanal**).
- Foi construído um **dicionário de dados** para auxiliar na compreensão semântica do dataset e servir como referência nas etapas subsequentes.

---

### Conclusão

O conjunto de dados apresenta **boa consistência estrutural e semântica**, sendo adequado para análise, desde que passe por:

- conversão e padronização de tipos,
- normalização de valores monetários e temporais,
- seleção e possível remoção de colunas pouco informativas.

Esses ajustes serão realizados na etapa de **Data Preparation**, garantindo um dataset limpo, coerente e pronto para análises e modelagens posteriores.